In [1]:
from pyper.viper_interface.viper_classes import PolhemusViper
from pyper.io.io_utils import extract_data_from_frame_continuous
from threading import Event
from multiprocessing.pool import ThreadPool
import time
import pandas as pd
import array

In [2]:
viper = PolhemusViper()
viper.connect()

In [3]:
viper.start_continuous()

[86, 80, 82, 67, 24, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 103, 246, 0, 0]


In [3]:
viper.stop_continuous()

[86, 80, 82, 80, 116, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 66, 8, 0, 25, 201, 206, 65, 46, 124, 103, 66, 66, 50, 160, 63, 214, 59, 18, 194, 85, 186, 151, 193, 124, 116, 3, 67, 0, 0, 0, 0, 1, 2, 0, 0, 29, 42, 228, 64, 32, 153, 118, 65, 206, 225, 32, 63, 9, 29, 242, 194, 142, 60, 60, 65, 51, 91, 31, 67, 0, 0, 0, 0, 2, 194, 3, 0, 157, 48, 6, 66, 55, 73, 233, 63, 139, 180, 143, 64, 145, 230, 8, 195, 209, 55, 70, 65, 80, 98, 12, 67, 0, 0, 0, 0, 29, 47, 0, 0]


In [5]:
viper.get_single_pno(pno_mode="standard")

,sensor_n,frames,time,x,y,z,azimuth,elevation,roll
0,1,13984,2022-05-09T01:19:51.225131,25.884882,57.852329,1.241916,-36.628555,-19.006105,131.447342
1,2,13984,2022-05-09T01:19:51.225131,7.130177,15.412402,0.628317,-121.056694,11.764623,159.357880
2,3,13984,2022-05-09T01:19:51.225131,33.547073,1.824214,4.491611,-136.895950,12.382915,140.390244


In [50]:

viper.get_single_pno()

,sensor_n,frames,time,x,y,z,azimuth,elevation,roll
0,1,508611,2022-05-09T01:07:50.370316,8.294545,51.357567,14.551523,123.009445,48.910873,-166.783478
1,2,508611,2022-05-09T01:07:50.370316,7.129799,15.412593,0.628341,-121.055176,11.763800,159.357513
2,3,508611,2022-05-09T01:07:50.370316,33.546608,1.822145,4.492939,-136.897614,12.394907,140.376556


In [27]:
max_cmd_read_size = 4 + 4 + 8 * 16 + 4
max_cmd_read_size

140

In [38]:
t = array.array('B', [86, 80, 82, 67, 24, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 95, 161, 0, 0])
viper.dev.write(0x02, t, 200)

resp = viper.dev.read(viper.endpoint, 200, 200)
print(resp)

array('B', [86, 80, 82, 67, 136, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 197, 198, 5, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 5, 0, 210, 173, 224, 65, 40, 110, 120, 66, 184, 25, 156, 63, 240, 150, 150, 66, 180, 248, 195, 191, 58, 70, 89, 194, 0, 0, 0, 0, 1, 2, 0, 0, 113, 39, 228, 64, 235, 153, 118, 65, 192, 250, 32, 63, 106, 28, 242, 194, 8, 58, 60, 65, 206, 90, 31, 67, 0, 0, 0, 0, 2, 194, 3, 0, 11, 48, 6, 66, 1, 120, 233, 63, 37, 173, 143, 64, 109, 227, 8, 195, 24, 90, 70, 65, 98, 100, 12, 67, 0, 0, 0, 0, 5, 229, 0, 0])


In [54]:
viper._write_and_read(t)

USBTimeoutError: [Errno 10060] Operation timed out

In [13]:
viper.dev.read(0x02, viper.conf['max_size'], 200)

USBTimeoutError: [Errno 10060] Operation timed out

error: required argument is not an integer

In [4]:

viper.start_continuous(viper.dev, viper.endpoint, pno_mode="standard", framecount="reset")

AttributeError: 'PolhemusViper' object has no attribute 'cmd_viper'

In [8]:
viper.stop_continuous(dev, ep)

[86, 80, 82, 67, 24, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 103, 246, 0, 0]


In [7]:
stop_con = Event()
pool = ThreadPool(processes=1)
async_result = pool.apply_async(viper.read_continuous, (dev, ep, stop_con))

time.sleep(30)


stop_con.set()
df = async_result.get()
# df = Thread(target=viper.read_continuous, args=(dev, ep, stop_con), daemon=True).start()

In [8]:
df1 = pd.DataFrame()
for i in range(len(df)):
    df2 = extract_data_from_frame_continuous(df[i])
    df1 = pd.concat([df1, df2], ignore_index=True)
# df1[30:100]

In [11]:
df1[5000:5020]

,sensor_n,frames,time,x,y,z,azimuth,elevation,roll
5000,1,22109,2022-05-06T12:44:33.770768,18.188828,5.734060,-3.234791,110.873169,9.454902,80.842033
5001,2,22109,2022-05-06T12:44:33.770768,42.936878,-37.987923,-1.470918,84.442154,6.991598,165.083130
5002,1,22111,2022-05-06T12:44:33.778769,18.188793,5.734065,-3.234854,110.873268,9.454646,80.842506
5003,2,22111,2022-05-06T12:44:33.778769,42.939579,-37.985188,-1.465786,84.445930,6.998866,165.078491
5004,1,22113,2022-05-06T12:44:33.788078,18.188763,5.734066,-3.234995,110.873383,9.454167,80.843658
5005,2,22113,2022-05-06T12:44:33.788078,42.939491,-37.984241,-1.463059,84.444633,7.004036,165.073059
5006,1,22115,2022-05-06T12:44:33.796227,18.188784,5.734043,-3.234959,110.873276,9.454263,80.843452
5007,2,22115,2022-05-06T12:44:33.796227,42.937660,-37.986572,-1.470663,84.443726,6.991900,165.084976
5008,1,22117,2022-05-06T12:44:33.804235,18.188780,5.734065,-3.234906,110.873360,9.454198,80.842957
5009,2,22117,2022-05-06T12:44:33.804235,42.940128,-37.983795,-1.466336,84.450104,6.998734,165.076157


In [3]:
viper.get_single_pno(dev=dev, ep=ep, pno_mode="standard")

error: unpack requires a buffer of 4 bytes

In [6]:
viper.start_continuous(dev, ep, pno_mode="standard", framecount="reset")

[86, 80, 82, 67, 24, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 103, 246, 0, 0]


In [4]:
dev.read(ep, 100, 200)

array('B', [86, 80, 82, 80, 84, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 66, 0, 0, 101, 231, 15, 65, 139, 200, 180, 64, 106, 101, 20, 64, 105, 254, 43, 67, 7, 125, 218, 193, 119, 110, 35, 195, 0, 0, 0, 0, 1, 130, 4, 0, 10, 166, 51, 66, 127, 109, 7, 194, 64, 72, 113, 62, 40, 135, 238, 66, 174, 8, 212, 192, 192, 81, 3, 67, 0, 0, 0, 0, 34, 165, 0, 0])

In [3]:
df = viper.get_single_pno(dev=dev, ep=ep, pno_mode="standard")
df

,sensor_n,frames,time,x,y,z,azimuth,elevation,roll
0,1,24154,2022-05-05T22:48:39.533252,9.010628,5.649452,2.351197,172.175293,-28.035522,-165.601578
1,2,24154,2022-05-05T22:48:39.533252,44.924061,-33.885765,0.258443,119.227028,-6.605873,131.344360
